
## README / Quick‑Start / Rotation Pipeline

**Directory layout expected**

```
rotation/
└── batches/
    ├── rotation_2025MMDD_01/          # <- renamed input folder
    │   ├── images/
    |   |   |__ boxes/(all crops)
    │   │   └── default/*.png
    │   └── annotations/
    │       └── instances_default.json
            |__ instances_updaated.json
    └── ...
```

> ⚠️ If your raw data are still in `rotation/batches/images/default`  
> run section **1 – Rename batches** first.



In [12]:
import cv2, os, math, sys
from pathlib import Path
from typing import List, Tuple, Dict, Any
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os, json, shutil, random, math, datetime as dt
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp

In [13]:

# Where am I?
print("Working dir :", Path.cwd())

# Show the absolute target
BATCHES_DIR = Path("../data/rotation/batches")               # or Path('/absolute/path/to/project')
print("Batch dir   :", BATCHES_DIR)

# Does it exist?
print("Exists?     :", BATCHES_DIR.exists())
print("Contents    :", list(BATCHES_DIR.iterdir())[:5])  # peek first 5 entries


Working dir : c:\Users\saschamueller\Documents\GitHub\ocr-rec-lab\pipeline
Batch dir   : ..\data\rotation\batches
Exists?     : True
Contents    : [WindowsPath('../data/rotation/batches/rotation_20250710_01'), WindowsPath('../data/rotation/batches/rotation_20250710_02'), WindowsPath('../data/rotation/batches/rotation_20250710_03'), WindowsPath('../data/rotation/batches/rotation_20250710_04'), WindowsPath('../data/rotation/batches/rotation_20250710_05')]


In [9]:

def rename_batches(batches_dir: Path, prefix: str = 'rotation', date_fmt: str = '%Y%m%d') -> None:
   
    today = dt.datetime.today().strftime(date_fmt)
    index = 1
    
    print(batches_dir.iterdir())
    print([p for p in batches_dir.iterdir()])
    
    unnamed = []

    
    for p in batches_dir.iterdir():
        print(f'P Name: {p.name}')
        print(f'P Type: {type(p.name)}')
        print(f'Prefix Type: {type(prefix)}')

        pname = p.name
        
        if (p.is_dir() and p.name not in ('images','annotations') and pname.find(prefix) == -1):
            unnamed.append(p)


    #unnamed = [p for p in batches_dir.iterdir() if p.is_dir() and p.name not in ('images','annotations') ]
    
    print(f'unnamed: {unnamed}')

    
    # also handle loose images/annotations sitting directly
    if (batches_dir/'images').exists() and (batches_dir/'annotations').exists():
        unnamed.append(batches_dir)
    if not unnamed:
        print('Nothing to rename – folders already structured ✔️')
        return
    for src in unnamed:
        target = batches_dir/f"{prefix}_{today}_{index:02d}"
        index += 1
        target.mkdir(exist_ok=True)
        for sub in ('images', 'annotations'):
            sub_path = src/sub
            if sub_path.exists():
                shutil.move(str(sub_path), target/ sub)
        # remove empty src folder if it wasn't batches_dir
        if src != batches_dir:
            try:
                src.rmdir()
            except OSError:
                pass
        print(f"Moved {src} -> {target}")


In [27]:
# Uncomment to execute
rename_batches(BATCHES_DIR)

<generator object Path.iterdir at 0x0000020AFF1CADC0>
[WindowsPath('../data/rotation/batches/task_batch 30_backup_2025_07_04_07_20_47_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 1.4_backup_2025_06_17_18_33_04_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 1.5_backup_2025_06_12_21_16_14_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 21_backup_2025_07_02_15_31_25_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 22_backup_2025_07_01_06_34_32_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 23_backup_2025_07_01_06_27_06_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 24_backup_2025_07_01_06_25_39_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 25_backup_2025_07_01_06_23_51_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 26_backup_2025_07_01_06_23_16_COCO'), WindowsPath('../data/rotation/batches/task_lyd batch 63_backup_2025_06_30_14_33_56_COCO'), WindowsPath('../data/rotation/batches

In [ ]:
# Optimized JSON and batch processing with caching
coco_cache = {}

def load_coco(json_path: Path) -> Dict[str, Any]:
    """Load COCO JSON with caching"""
    if json_path in coco_cache:
        return coco_cache[json_path]
    
    with open(json_path, 'r', encoding='utf-8') as f:
        coco = json.load(f)
    coco_cache[json_path] = coco
    return coco

def create_obb_tuple(anns):
    bbox = anns.get("bbox", "No bbox found")
    if len(bbox) == 4:    
        x, y, w, h = anns["bbox"]
        cx = x + (w/2)
        cy = y + (h/2)
        angle = anns["attributes"].get("rotation", 0.0)
        obb_list = [cx, cy, w, h, angle]
        anns["bbox"] = obb_list
    else: 
        print("Weirdle after every element is on 5 tuples it starts to iterate again")

def process_single_batch(batch_path: Path):
    """Process a single batch for OBB conversion"""
    json_path = batch_path / "annotations" / "instances_default.json"
    if not json_path.exists():
        return
    
    coco = load_coco(json_path)
    
    for anns in coco['annotations']:
        create_obb_tuple(anns)
        
    output_path = batch_path / "annotations" / "instances_updated.json"
    with open(output_path, 'w') as f:
        json.dump(coco, f)
    
    print(f"Processed: {batch_path.name}")

def convert_all_batches():
    """Convert all batches in parallel"""
    batch_paths = [p for p in BATCHES_DIR.iterdir() if "rotation" in p.name]
    
    with ThreadPoolExecutor(max_workers=min(len(batch_paths), mp.cpu_count())) as executor:
        futures = [executor.submit(process_single_batch, p) for p in batch_paths]
        
        for future in tqdm(futures, desc="Converting batches"):
            future.result()

convert_all_batches()

In [29]:
def crop_oriented_bbox(img, cx, cy, w, h, theta):
    # Step 1: Rotate the entire image around the bbox center
    M = cv2.getRotationMatrix2D((cx, cy), theta, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    
    # Step 2: Crop the now-aligned rectangle
    x1 = int(cx - w/2)
    y1 = int(cy - h/2)
    x2 = int(cx + w/2)
    y2 = int(cy + h/2)
    
    # Ensure bounds are within image
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(img.shape[1], x2), min(img.shape[0], y2)
    
    cropped = rotated[y1:y2, x1:x2]
    return cropped

In [ ]:
# Ultra-fast parallel box cropping with live progress tracking
from concurrent.futures import as_completed

def process_image_group(args):
    """Process a single image and all its annotations"""
    image_path, file_name, annotations, dest_dir = args
    
    img_path = image_path / file_name
    img_arr = cv2.imread(str(img_path))
    if img_arr is None:
        return f"Could not load {img_path}"
    
    file_number = file_name.replace('.png', "")
    crops_made = 0
    
    # Process all annotations for this image
    for ann in annotations:
        try:
            cx, cy, w, h, theta = ann["bbox"]
            rotated_box = crop_oriented_bbox(img_arr, cx, cy, w, h, theta)
            
            output_path = dest_dir / f"{file_number}_{ann['id']}.png"
            cv2.imwrite(str(output_path), rotated_box)
            crops_made += 1
        except Exception as e:
            continue
    
    return f"Processed {file_name}: {crops_made} crops"

def crop_boxes_from_batch(batch_path: Path):
    """Process box cropping for a single batch with live progress"""
    if not ("rotation" in batch_path.name and 
            (batch_path / "annotations" / "instances_updated.json").exists() and 
            (batch_path / "images").exists()):
        return f"Skipped {batch_path.name} - missing files"
    
    image_path = batch_path / "images" / "default"
    coco = load_coco(batch_path / "annotations" / "instances_updated.json")
    
    DEST_IMG_DIR = batch_path / "images" / "boxes"
    DEST_IMG_DIR.mkdir(parents=True, exist_ok=True)
    
    # Group annotations by image_id
    image_groups = {}
    for ann in coco["annotations"]:
        image_id = ann["image_id"]
        if image_id not in image_groups:
            image_groups[image_id] = []
        image_groups[image_id].append(ann)
    
    # Prepare arguments for parallel processing
    process_args = []
    for image_id, annotations in image_groups.items():
        img_meta = next((img for img in coco["images"] if img["id"] == image_id), None)
        if not img_meta:
            continue
        
        file_name = img_meta.get('file_name')
        process_args.append((image_path, file_name, annotations, DEST_IMG_DIR))
    
    # Process images in parallel with live progress
    max_workers = min(len(process_args), mp.cpu_count())
    processed_count = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        futures = {executor.submit(process_image_group, args): args for args in process_args}
        
        # Process with live progress
        with tqdm(total=len(process_args), desc=f"Processing {batch_path.name}", 
                 unit="images", leave=False) as pbar:
            for future in as_completed(futures):
                result = future.result()
                processed_count += 1
                pbar.update(1)
                pbar.set_postfix({"completed": processed_count})
    
    return f"✓ {batch_path.name}: {processed_count} images, {len(coco['annotations'])} crops"

def crop_all_boxes_with_progress():
    """Process all batches with nested progress bars"""
    batch_paths = [p for p in BATCHES_DIR.iterdir() if "rotation" in p.name]
    
    # Use ThreadPoolExecutor instead of ProcessPoolExecutor for better progress tracking
    max_batch_workers = min(len(batch_paths), 4)
    
    with ThreadPoolExecutor(max_workers=max_batch_workers) as executor:
        # Submit all batch tasks
        futures = {executor.submit(crop_boxes_from_batch, p): p for p in batch_paths}
        
        # Process with main progress bar
        with tqdm(total=len(batch_paths), desc="Processing batches", 
                 unit="batch", position=0) as main_pbar:
            for future in as_completed(futures):
                batch_path = futures[future]
                result = future.result()
                main_pbar.update(1)
                main_pbar.set_postfix({"current": batch_path.name})
                print(f"  {result}")

# Run with live progress
crop_all_boxes_with_progress()

In [11]:
import cv2
import numpy as np

def rotate_patch(patch, angle):
    h, w = patch.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    cos, sin = abs(M[0,0]), abs(M[0,1])
    new_w, new_h = int(h*sin + w*cos), int(h*cos + w*sin)
    M[0,2] += new_w/2 - w/2
    M[1,2] += new_h/2 - h/2

    # Prüfen ob Bild einen Alphakanal hat
    if patch.shape[2] == 4:
        # Transparenz beibehalten
        rotated = cv2.warpAffine(
            patch,
            M,
            (new_w, new_h),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0, 0)  # Transparenter Hintergrund
        )
    else:
        # Kein Alphakanal → normal mit weißem Hintergrund
        rotated = cv2.warpAffine(
            patch,
            M,
            (new_w, new_h),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(255, 255, 255)
        )

    return rotated


In [ ]:
# Optimized rotation with live progress tracking and parallel processing
ANGLES: List[int] = [0, 90, 180, 270]

def process_single_image_rotation(img_path):
    """Process rotations for a single image"""
    img_arr = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img_arr is None:
        return f"Could not load {img_path}"
    
    box_nr = img_path.stem
    boxes_dir = img_path.parent
    rotations_made = 0
    
    # Process all rotations for this image
    for angle in ANGLES:
        try:
            rotated_box = rotate_patch(img_arr, angle)
            out_path = boxes_dir / f"{box_nr}_{angle}.png"
            cv2.imwrite(str(out_path), rotated_box)
            rotations_made += 1
        except Exception as e:
            continue
    
    return f"Rotated {img_path.name}: {rotations_made} angles"

def rotate_images_in_batch(batch_path: Path):
    """Rotate all images in a single batch with live progress"""
    if 'rotation' not in batch_path.name:
        return f"Skipped {batch_path.name} - not a rotation batch"
    
    boxes_dir = batch_path / 'images' / 'boxes'
    if not boxes_dir.exists():
        return f"Skipped {batch_path.name} - no boxes directory"
    
    # Get all image files and filter out already rotated images
    image_files = list(boxes_dir.glob("*.png"))
    base_images = [f for f in image_files if not any(f.stem.endswith(f"_{angle}") for angle in ANGLES)]
    
    if not base_images:
        return f"No base images found in {batch_path.name}"
    
    # Process images in parallel with live progress
    max_workers = min(len(base_images), mp.cpu_count())
    processed_count = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        futures = {executor.submit(process_single_image_rotation, img_path): img_path for img_path in base_images}
        
        # Process with live progress
        with tqdm(total=len(base_images), desc=f"Rotating {batch_path.name}", 
                 unit="images", leave=False) as pbar:
            for future in as_completed(futures):
                result = future.result()
                processed_count += 1
                pbar.update(1)
                pbar.set_postfix({"completed": processed_count})
    
    total_rotations = processed_count * len(ANGLES)
    return f"✓ {batch_path.name}: {processed_count} images → {total_rotations} rotated versions"

def rotate_all_batches_with_progress():
    """Rotate all batches with nested progress bars"""
    batch_paths = [p for p in BATCHES_DIR.iterdir() if 'rotation' in p.name]
    
    # Use ThreadPoolExecutor for better progress tracking
    max_batch_workers = min(len(batch_paths), 4)
    
    with ThreadPoolExecutor(max_workers=max_batch_workers) as executor:
        # Submit all batch tasks
        futures = {executor.submit(rotate_images_in_batch, p): p for p in batch_paths}
        
        # Process with main progress bar
        with tqdm(total=len(batch_paths), desc="Rotating batches", 
                 unit="batch", position=0) as main_pbar:
            for future in as_completed(futures):
                batch_path = futures[future]
                result = future.result()
                main_pbar.update(1)
                main_pbar.set_postfix({"current": batch_path.name})
                print(f"  {result}")

# Run with live progress
rotate_all_batches_with_progress()

Rotating batches:   0%|          | 0/13 [00:00<?, ?batch/s]














































































































































































































































































































































































































































































































































































































































































































































































































































































































































































  ✓ rotation_20250710_03: 4143 images → 16572 rotated versions


  ✓ rotation_20250710_02: 4473 images → 17892 rotated versions


Rotating rotation_20250710_04:  20%|█▉        | 4382/22068 [02:50<12:51, 22.93images/s, completed=4382]


































































































































































































































































































































































































































































































































































































































































































































































































































































































































  ✓ rotation_20250710_01: 21317 images → 85268 rotated versions


Rotating rotation_20250710_05:  80%|███████▉  | 17240/21552 [06:43<02:07, 33.87images/s, completed=17240]
































































































































































































































































































































































































































































































































































































































































































































































































































































































































## Alle Klassen in train/test umschreiben 

In [18]:
# Optimized class organization with batch processing
import random
from typing import List
# Show the absolute target
BATCHES_DIR = Path("../data/rotation/batches") 
ANGLES: List[int] = [0, 90, 180, 270]


def organize_into_classes(dataset_path, out_base_path, train_ratio=0.8):
    """Organize images into train/test splits with optimized file operations"""
    dataset_path = Path(dataset_path)
    out_base_path = Path(out_base_path)

    # Create all directories at once
    for split in ['train', 'test']:
        for angle in ANGLES:
            (out_base_path / split / str(angle)).mkdir(parents=True, exist_ok=True)

    # Group files by angle for batch processing
    angle_files = {angle: [] for angle in ANGLES}
    
    for img_file in dataset_path.glob("*.png"):
        for angle in ANGLES:
            if f"_{angle}.png" in img_file.name:
                angle_files[angle].append(img_file)
                break
    
    # Process each angle group
    for angle, files in angle_files.items():
        if not files:
            continue
            
        # Shuffle once for consistent random split
        random.shuffle(files)
        split_idx = int(len(files) * train_ratio)
        
        train_files = files[:split_idx]
        test_files = files[split_idx:]
        
        # Batch copy operations
        for file_list, split in [(train_files, 'train'), (test_files, 'test')]:
            target_dir = out_base_path / split / str(angle)
            for img_file in tqdm(file_list, desc=f"Copying {split} {angle}°"):
                shutil.copy2(img_file, target_dir / img_file.name)

def organize_all_batches(out_base_path, train_ratio=0.8):
    """Organize all batches into classification structure"""
    out_base_path = Path(out_base_path)
    
    # Process all batches
    batch_paths = [p for p in BATCHES_DIR.iterdir() if 'rotation' in p.name]
    
    for batch_path in tqdm(batch_paths, desc="Organizing batches"):
        boxes_dir = batch_path / 'images' / 'boxes'
        if boxes_dir.exists():
            organize_into_classes(boxes_dir, out_base_path, train_ratio)

In [20]:
# Use the optimized function for all batches
organize_all_batches(
    out_base_path=Path("../data/rotation/classification"),
    train_ratio=0.9
)

Organizing batches:   0%|          | 0/13 [00:02<?, ?it/s]


KeyboardInterrupt: 